# install sci-kit learn and restart runtime

In [ ]:
!pip install scikit-learn==1.2.2
!pip install imblearn

#setup:
- connect to google drive in this step

In [ ]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow
import random

In [ ]:
#Importing essential libraries
import matplotlib.pyplot as plt
from statistics import mean
from matplotlib import pyplot
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
project_path = '/content/drive/MyDrive/230702/demo-me-2021-07-14'

# Load classifier model:
requires classifier file name

In [ ]:
classifier_name = input('Enter the classifier name: ')
print(f'You entered {classifier_name}')

# open a file, where you stored the pickled data
file = open(project_path+ '/classifiers/' + classifier_name+ '.pickle', 'rb')

# dump information to that file
RFC = pickle.load(file)
print(RFC)

# close the file
file.close()

Enter the classifier name: 230705_2nine
You entered 230705_2nine
RandomForestClassifier(n_estimators=10, random_state=5)


# Get video features:

In [ ]:
# load an all_i_features pickle
video_name = '221002_PZ71_1'
with open(project_path + "/behaviors/" + video_name + "_features.pickle", 'rb') as file:
    all_i_features = pickle.load(file)

# Forest features data:

In [ ]:
input_features = all_i_features
print(input_features.shape)

(36000, 9)


In [ ]:
print(np.any(np.isnan(input_features)))
print(np.any(np.isinf(input_features)))

False
False


In [ ]:
#input_features[input_features < -10000] = -10000 # should all be positive though
#input_features[input_features > 10000] = 10000

In [ ]:
print(np.max(input_features))
print(np.min(input_features))

298.3886965218354
0.0


#Run RFC and turn y_pred into timestamps of changes

In [ ]:
y_pred = RFC.predict(input_features)

In [ ]:
# smooth over 1s (30Hz frames)
# for each second, only consider those frames
sm_pred=np.zeros((int(len(y_pred) / 30), 1))
for i in range(30, len(y_pred), 30):
  sm_pred[int(i/30)] = np.median(y_pred[i-30:i])

print(sm_pred.shape)

(1200, 1)


In [ ]:
behavior_pred = np.empty((1,2))
prior_behavior = 0
prior_behavior_start = 0
for i in range(1, len(sm_pred)):
  if sm_pred[i] != sm_pred[i-1] and sm_pred[i-1] == 1:
    duration = i - prior_behavior_start # duration in secs
    #if duration > 1:
    behavior_pred = np.vstack([behavior_pred, [prior_behavior_start*30, duration]]) # [frame, secs duration]
    prior_behavior = sm_pred[i]

  elif sm_pred[i] != sm_pred[i-1] and sm_pred[i] == 1:
    prior_behavior_start = i

print(behavior_pred.shape)

(40, 2)


In [ ]:
fname = project_path + '/behaviors/'+video_name+'_pred.xlsx'
if os.path.isfile(fname):
  print('already a file')
else:
  behavior_pred_df = pd.DataFrame(behavior_pred[1:, :], columns=['Start Frame', 'Duration (s)'])
  behavior_pred_df.to_excel(fname, index=False)

# get labeled behavior video

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture(project_path + '/videos/' + video_name + 'DLC_resnet50_demoJul14shuffle1_50000_full.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)

fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
out = cv2.VideoWriter(project_path + '/behaviors/'+video_name+'_rfc.mp4',fourcc, 30, size)

i = 0;
max_frames = 36000
while(True):
  if i >= max_frames:
    break
  ret, frame = cap.read()

  font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(frame,
              str(i),
              (150, 20),
              font, 0.5,
              (0, 0, 0),
              2,
              cv2.LINE_4)

  text = 'none'
  label = sm_pred[int(i/30)]
  if label == 1:
    text = 'Mount'
    cv2.putText(frame,
              text,
              (20, 20),
              font, 1,
              (0, 255, 0.75),
              2,
              cv2.LINE_4)
  elif label == 0:
    text = 'Nonmount'
    cv2.putText(frame,
                text,
                (20, 20),
                font, 0.75,
                (0, 0, 0),
                2,
                cv2.LINE_4)
  out.write(frame)
  i = i+1

  # creating 'q' as the quit
  # button for the video
  if cv2.waitKey(1) & 0xFF == ord('q'):
      break

cap.release()

out.release()
# close all windows
cv2.destroyAllWindows()

# look at individual features data

In [ ]:
#print(input_features)
input_features_df = pd.DataFrame(input_features)
input_features_df.to_excel(project_path + '/input_features.xlsx', index=False)